In [174]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
# Import comet_ml in the top of your file
from comet_ml import Experiment
##Needs to be imported before sklearn
from sklearn.model_selection import train_test_split
import sklearn

import torch
import torch.utils.data
from torch.utils.data import Dataset, TensorDataset, DataLoader #as DataLoaderBase
from torch import nn, optim, sigmoid
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.nn import modules

from torch.autograd import Variable

#from torchaudio import transforms
#from torchaudio import Datasets

import os
import sys
import shutil
from glob import glob
import datetime
import re
from tqdm import tqdm
import time


#from torchviz import make_dot, make_dot_from_trace


In [175]:
# Check device type


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [176]:
device


device(type='cpu')

In [177]:
def data_creator(T = 20, L = 10000, N = 1000):
    np.random.seed(2)



    x = np.empty((N, L), 'int64')
    x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
    data = np.sin(x / 1.0 / T).astype('float64')
    data = sklearn.preprocessing.normalize(data)

    torch.save(data, open('traindata.pt', 'wb'))

In [178]:
# input to LSTM is sequence length, batch, input_size   
#sequence length ---- how many time steps RNN
#batch 


In [179]:
class EncoderRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_dim
        self.hidden_size = hidden_dim
        hidden_size = hidden_dim
        self.n_layers = n_layers
        self.dropout = dropout
        

        self.gru = nn.GRU(hidden_size, input_size, n_layers, dropout=self.dropout, bidirectional=True)
        
    def forward(self, input_seqs, input_lengths = 22000, hidden=None):
        # Note: we run this all at once (over multiple batches of multiple sequences)

        packed = torch.nn.utils.rnn.pack_padded_sequence(input_seqs, input_lengths)
        outputs, hidden = self.gru(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs
        return outputs, hidden

In [180]:
class RAE_model(nn.Module):
    def __init__(self, INPUT_SIZE, HIDDEN_SIZE):
        super(RAE_model, self).__init__()

        self.rnn1 = nn.modules.GRU( INPUT_SIZE, HIDDEN_SIZE, bias = True)
        self.rnn2 = nn.modules.GRU( HIDDEN_SIZE, 100, bias = True)
        self.fc21 = nn.Linear(100, 20)
        self.fc22 = nn.Linear(100, 20)
        self.fc3 = nn.Linear(20, 100)
        self.rnn2 = nn.modules.GRU( 100, HIDDEN_SIZE, bias = True)
        self.rnn2 = nn.modules.GRU(HIDDEN_SIZE, INPUT_SIZE, bias = True)

    def encode(self, x):
        h1 = F.relu(self.rnn1((20,x)))
        h2 = F.relu(self.rnn2(h1))
        return self.fc21(h2), self.fc22(h2)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        h4 = F.relu(self.rnn3(h3))
        return sigmoid(self.rnn4(h4))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar



In [181]:
class RVAE_model(nn.Module):
    """
    Recurrent VAE Model with GRUcells sequence length needs to be added
    """
    def __init__(self):
        super(RVAE_model, self).__init__()

        self.rnn1 = nn.modules.GRUCell(1000, 400, bias = True)
        self.rnn2 = nn.modules.GRUCell(400, 100, bias = True)
        self.fc21 = nn.Linear(100, 20)
        self.fc22 = nn.Linear(100, 20)
        self.fc3 = nn.Linear(20, 100)
        self.rnn2 = nn.modules.GRUCell(100, 400, bias = True)
        self.rnn2 = nn.modules.GRUCell(400, 1000, bias = True)

    def encode(self, x):
        h1 = F.relu(self.rnn1(x))
        h2 = F.relu(self.rnn2(h1))
        return self.fc21(h2), self.fc22(h2)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        h4 = F.relu(self.rnn3(h3))
        return sigmoid(self.rnn4(h4))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar



In [182]:
class VAE_model(nn.Module):
    def __init__(self):
        super(VAE_model, self).__init__()

        self.fc1 = nn.Linear(1000, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 1000)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        """
        Developed by 
        
        """
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

In [183]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.fc1 = nn.Linear(1000, 400)
        self.fc2 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 1000)
        
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return F.relu(self.fc2(h1))
        
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.relu(self.fc4(h3))
    
    def forward(self, x):
        z = self.encode(x)
        return self.decode(z)
        

In [2]:
class sineDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, seq_len, overlap):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = torch.load('traindata.pt')
        self.label = []
        vals = len(self.data)
        for val in range(vals):
            self.label.append(val)
        if torch.cuda.is_available():
            self.data, self.label = map(torch.cuda.FloatTensor, (self.data, self.label))
        else:
            self.data, self.label = map(torch.FloatTensor, (self.data, self.label))

        self.data = window_mask(data, seq_len, overlap)
        print("shape = ", self.data.shape)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]
    
    def window_mask(x, seq_len, overlap):
        data = []
        results = []
        x = torch.tensor(x)
        start_val = 0
        x_t = torch.transpose(x, 0, 1)
        occurences = int(len(x)/(seq_len*overlap))
        print(occurences)
        for rows in range(len(x)):

            start_val = 0
            for i in range(occurences-1):
                    value = x[rows,start_val:(start_val+seq_len)]
                    #print('value = ', value)
                    start_val += int(seq_len*(1 - overlap))
                    data.append(value.unsqueeze(0))
                    #data.append(value)


            var = torch.cat(data)        
            results.append(var.unsqueeze(0))

            data = []
        new_res = torch.cat(results)
        print(new_res.shape)
        #print("new_res = ", new_res)

        return new_res
    
        

NameError: name 'Dataset' is not defined

In [185]:
class data_loader(DataLoaderBase):

    def __init__(self, dataset, batch_size, seq_len, overlap_len,
                 *args, **kwargs):
        super().__init__(dataset, batch_size, *args, **kwargs)
        self.seq_len = seq_len
        self.overlap_len = overlap_len

    def __iter__(self):
        for batch in super().__iter__():
            (batch_size, n_samples) = batch.size()

            reset = True

            for seq_begin in range(self.overlap_len, n_samples, self.seq_len):
                from_index = seq_begin - self.overlap_len
                to_index = seq_begin + self.seq_len
                sequences = batch[:, from_index : to_index]
                input_sequences = sequences[:, : -1]
                target_sequences = sequences[:, self.overlap_len :].contiguous()

                yield (input_sequences, reset, target_sequences)

                reset = False

    def __len__(self):
        raise NotImplementedError()

In [186]:

def get_model():
    
    #model = EncoderRNN(input_dim= input_dim, hidden_dim= hidden_dim).to(device)
    model = RAE_model(INPUT_SIZE= 1000, HIDDEN_SIZE= 400).to(device)

    
    return model, optim.Adam(model.parameters(), lr = lr)


In [187]:
def get_data(train_dataset, test_dataset, bs):
    return (DataLoader(train_dataset, batch_size = batch_size),
    DataLoader(test_dataset, batch_size = batch_size * 2))

In [188]:
###Need data_creator function

data_creator()

In [189]:
hidden_dim = 1000
latent = 5
input_dim = 5
epochs = 1000
seq_len = 32
batch_size = 32
input_size = 1000

overlap = .5
features = 997
latent =50


lr = 1e-4

In [190]:
 
full_dataset = sineDataset( )
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
train_dl, test_dl = get_data(train_dataset, test_dataset, batch_size)
print(train_size)

shape =  torch.Size([1000, 10000])
800


In [191]:
def loss_function(recon_batch, mu, logvar, x):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    MSE = F.binary_cross_entropy(recon_batch, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return MSE + KLD

def loss_function(output, x):
    return F.binary_cross_entropy(output, x)

In [196]:
def loss_batch(model, loss_function, xb, opt=None):
    recon_batch, mu,logvar = model(xb)
    loss = loss_function(recon_batch, mu,logvar, xb)
   
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    return loss.item(), len(xb)

def loss_batch(model, loss_function, xb, opt=None):
    output = model(xb)
    loss = loss_function(output, xb)
   
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    return loss.item(), len(xb)

In [215]:
def fit(epochs, model, loss_func, opt, train_dl, test_dl):
    for epoch in tqdm(range(epochs)):
        model.train()
        
        for xb in enumerate(train_dl):
            

            #xb = xb.reshape(1,batch_size,input_size).to(device)  
           
            if torch.cuda.is_available():
                xb = Variable(xb)
            else:
                xb = Variable(xb)
            
            """seq_len, batch_size, input_size
            input_size is 1000 seperate strings.
            batch_size = 16
            seq_len can be anything....
            
            
            """
            loss_batch(model, loss_func, xb, opt)
            model.eval()
            with torch.no_grad():
                losses,nums = zip(*[loss_batch(model, loss_func, xb)
                                    for xb in test_dl])
            val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)
        print(epoch, val_loss)

## Full training in 3 lines of code

In [216]:

model, opt = get_model()
#x = torch.randn(1, 10000)
#make_dot(model(x), params=dict(model.named_parameters()))

       
    

In [217]:
fit(epochs, model, loss_function, opt, train_dl, test_dl)

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]


TypeError: Variable data has to be a tensor, but got tuple

kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}   - need look this ---https://vxlabs.com/2017/12/08/variational-autoencoder-in-pytorch-commented-and-annotated/